In [1]:
import pandas as pd 
import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
# class LSTMFeatureExtractor(nn.Module):
#     def __init__(self, input_dim, hidden_dim, num_layers):
#         super(LSTMFeatureExtractor, self).__init__()
#         self.hidden_dim = hidden_dim
#         self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

#     def forward(self, x):
#         # x shape: (batch, seq_len, features)
#         lstm_out, (hn, cn) = self.lstm(x)  # lstm_out shape: (batch, seq_len, hidden_dim)
#         return torch.squeeze(lstm_out)


# class LSTMFeatureExtractor(nn.Module):
#     def __init__(self, input_dim, hidden_dim, num_layers):
#         super(LSTMFeatureExtractor, self).__init__()
#         self.hidden_dim = hidden_dim
#         self.num_layers = num_layers
#         self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        
#     def forward(self, x):
#         # x shape: (batch, seq_len, features)
#         _, (hn, _) = self.lstm(x)  # hn shape: (num_layers, batch, hidden_dim)
        
#         # Take the last layer's hidden state
#         hn_last_layer = hn[-1, :, :]  # Shape: (batch, hidden_dim)
        
#         # If you specifically need the output to be 2048 dimensions but your hidden_dim is different,
#         # you might consider adding a linear layer to map hidden_dim to 2048.
#         # Ensure hidden_dim is set to 2048 if you want the output directly without transformation.
#         return torch.squeeze(hn_last_layer)


# class LSTMFeatureExtractorWithPooling(nn.Module):
#     def __init__(self, input_dim, hidden_dim, num_layers):
#         super(LSTMFeatureExtractorWithPooling, self).__init__()
#         self.hidden_dim = hidden_dim
#         self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        
#     def forward(self, x):
#         # x shape: (batch, seq_len, features)
#         lstm_out, _ = self.lstm(x)  # lstm_out shape: (batch, seq_len, hidden_dim)
        
#         # Apply mean pooling across timesteps
#         # lstm_out shape after mean: (batch, hidden_dim)
#         mean_pooled = torch.mean(lstm_out, dim=1)
        
#         return torch.squeeze(mean_pooled)

# input_dim_vis = 2048  
# input_dim_aud = 2048  
# hidden_dim = 2048  
# num_layers = 1    

# feature_extractor_vis = LSTMFeatureExtractor(input_dim_vis, hidden_dim, num_layers)

In [3]:
# class EmotionDataset(Dataset):
#     def __init__(self, X, y):
#         self.X = X
#         self.y = y

#     def __len__(self):
#         return len(self.X)

#     def __getitem__(self, idx):
#         return self.X[idx], self.y[idx]

# def mean_pooling(batch_features_padded):
#     return torch.mean(batch_features_padded, dim=1)

# class FeatureDataset(Dataset):
#     def __init__(self, dataframe, feature_type, base_path="../data/"):
#         """
#         feature_type: 'visual_features' or 'audio_features'
#         """
#         self.dataframe = dataframe
#         self.feature_type = feature_type
#         self.base_path = base_path

#     def __len__(self):
#         return len(self.dataframe)

#     def __getitem__(self, idx):
#         file_path = self.dataframe.iloc[idx][self.feature_type]
#         features = np.load(self.base_path + file_path)
#         return torch.tensor(features, dtype=torch.float)
# def collate_fn(batch):
#     batch_features = [item for item in batch]
#     batch_features_padded = pad_sequence(batch_features, batch_first=True)
#     return mean_pooling(batch_features_padded)
# # Emotion Classifier Model
# class VisualClassifier(nn.Module):
#     def __init__(self):
#         super(VisualClassifier, self).__init__()
#         self.fc1 = nn.Linear(2048, 1024)
#         self.bn1 = nn.BatchNorm1d(1024)
#         self.dropout1 = nn.Dropout(0.5)
#         self.fc2 = nn.Linear(1024, 256)
#         self.bn2 = nn.BatchNorm1d(256)
#         self.dropout2 = nn.Dropout(0.5)
#         self.fc4 = nn.Linear(256, 4)  # Assuming 4 classes

#     def forward(self, x):
#         x = F.relu(self.bn1(self.fc1(x)))
#         x = self.dropout1(x)
#         x = F.relu(self.bn2(self.fc2(x)))
#         x = self.dropout2(x)
#         x = self.fc4(x)
#         return x
# def calculate_accuracy(model, data_loader):
#     model.eval()
#     correct, total = 0, 0
#     with torch.no_grad():
#         for inputs, labels in data_loader:
#             outputs = model(inputs)
#             _, predicted = torch.max(outputs, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()
#     return correct / total

In [4]:
class EmotionDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [5]:
def mean_pooling(batch_features_padded):
    return torch.mean(batch_features_padded, dim=1)

In [6]:
class FeatureDataset(Dataset):
    def __init__(self, dataframe, feature_type, base_path="../data/"):
        """
        feature_type: 'visual_features' or 'audio_features'
        """
        self.dataframe = dataframe
        self.feature_type = feature_type
        self.base_path = base_path

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        file_path = self.dataframe.iloc[idx][self.feature_type]
        features = np.load(self.base_path + file_path)
        return torch.tensor(features, dtype=torch.float)

In [7]:
def collate_fn(batch):
    batch_features = [item for item in batch]
    batch_features_padded = pad_sequence(batch_features, batch_first=True)
    return mean_pooling(batch_features_padded)

In [8]:
# Emotion Classifier Model
class VisualClassifier(nn.Module):
    def __init__(self):
        super(VisualClassifier, self).__init__()
        self.fc1 = nn.Linear(2048, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.dropout1 = nn.Dropout(0.7)
        self.fc2 = nn.Linear(256, 4)
        self.bn2 = nn.BatchNorm1d(4)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.fc2(x)))
        return x

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TextClassifier(nn.Module):
    def __init__(self, input_dim=768, output_dim=4, hidden_dim=256):
        super(TextClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.bn2 = nn.BatchNorm1d(hidden_dim // 2)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(hidden_dim // 2, output_dim)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x


In [10]:
# Emotion Classifier Model
class AudialClassifier(nn.Module):
    def __init__(self):
        super(AudialClassifier, self).__init__()
        self.fc1 = nn.Linear(128, 16)
        self.bn1 = nn.BatchNorm1d(16)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(16, 4)
        self.bn2 = nn.BatchNorm1d(4)
   
    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.fc2(x)))
        return x

In [11]:
def calculate_accuracy(model, data_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

In [12]:
def train_model(model, dataloaders, optimizer, criterion, device, num_epochs=50, patience=10):

    """
    Trains and validates the model.
    
    Args:
    - model (torch.nn.Module): The PyTorch model to train.
    - dataloaders (dict): A dictionary containing 'train' and 'val' DataLoaders.
    - optimizer (torch.optim.Optimizer): The optimizer to use for training.
    - criterion (torch.nn.Module): The loss function.
    - num_epochs (int): The number of epochs to train for.
    - patience (int): The patience for early stopping.
    """
    
    best_val_loss = float('inf')
    patience_counter = 0
    model = model.to(device)
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        for inputs, labels in dataloaders['train']:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        train_accuracy = calculate_accuracy(model, dataloaders['train'])
        
        # Validation phase
        val_loss = 0.0
        model.eval()
        with torch.no_grad():
            for inputs, labels in dataloaders['val']:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
        val_accuracy = calculate_accuracy(model, dataloaders['val'])
        val_loss /= len(dataloaders['val'])
        
        # Early Stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered")
                break
        
    print(f'Epoch {epoch+1}, Train Loss: {running_loss/len(dataloaders["train"]):.4f}, '
              f'Train Accuracy: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')
    return val_accuracy


In [13]:
def extract_and_pool_features(df, feature_types, base_path="../data/"):

    """
    Extracts features from specified columns in the DataFrame, applies mean pooling,
    and updates the DataFrame with new columns for these processed features.
    
    Args:
    - df (DataFrame): The pandas DataFrame containing the features.
    - feature_types (dict): A dictionary mapping from 'visual' and 'audio' to their respective column names in df.
    - base_path (str): Base path where the feature files are stored.
    """
    
    for key, column in feature_types.items():
        pooled_features = []
        for _, row in df.iterrows():
            file_path = row[column]
            features = np.load(f"{base_path}{file_path}")
            pooled_feature = np.mean(features, axis=0)
            pooled_features.append(pooled_feature)
        
        # Update DataFrame with new columns for pooled features
        df[f'extracted_{key}_features'] = pooled_features




In [14]:
def prepare_datasets_and_loaders(df, feature_columns, label_column='emotion_labels', batch_size=4, test_size=0.2):
    
    """
    Prepares datasets and dataloaders for training and validation.
    
    Args:
    - df (DataFrame): The pandas DataFrame containing the pooled features and labels.
    - feature_columns (list): List of column names for the features to be used.
    - label_column (str): The column name where the label data is stored.
    - batch_size (int): Batch size for the dataloaders.
    - test_size (float): Proportion of the dataset to include in the test split.
    
    Returns:
    - A dictionary of dataloaders for training and validation for each feature type.
    """

    dataloaders = {}
    y = torch.tensor(df[label_column].values, dtype=torch.long)

    for feature_type in feature_columns:
        X = np.array(df[feature_type].tolist(), dtype=np.float32)
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size, random_state=42)
        
        train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train), y_train)
        val_dataset = torch.utils.data.TensorDataset(torch.tensor(X_val), y_val)
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        
        dataloaders[f'{feature_type}_train'] = train_loader
        dataloaders[f'{feature_type}_val'] = val_loader

    return dataloaders



### RUN CLASSIFIER

In [15]:
df = pd.read_csv('../data/csv/dataset.csv')

model_aud = AudialClassifier()
model_aud = model_aud.to(device)

model_vis = VisualClassifier()
model_vis = model_vis.to(device)

model_text = TextClassifier()

criterion = nn.CrossEntropyLoss()

feature_types = {'visual': 'visual_features', 'audio': 'acoustic_features'}#, 'text':'text_features'}
extract_and_pool_features(df, feature_types)

feature_columns = ['extracted_visual_features', 'extracted_audio_features']
dataloaders = prepare_datasets_and_loaders(df, feature_columns)

optimizer_aud = optim.Adam(model_aud.parameters(), lr=0.001, weight_decay=1e-4)
optimizer_vis = optim.Adam(model_vis.parameters(), lr=0.001, weight_decay=1e-4)

models_optimizers = {
    'extracted_visual_features': (model_vis, optimizer_vis),
    'extracted_audio_features': (model_aud, optimizer_aud),
}

# Train each model with its corresponding feature type
for feature_type, (model, optimizer) in models_optimizers.items():
    print(f"Training with {feature_type}:")
    break
    train_model(model, 
                {'train': dataloaders[f'{feature_type}_train'], 'val': dataloaders[f'{feature_type}_val']}, 
                optimizer, criterion)


Training with extracted_visual_features:


### PARAM GRID SEARCH

In [16]:
from itertools import product
import torch.optim as optim
import torch.nn as nn

feature_columns = {
    'extracted_visual_features'
}
# Define your grid of hyperparameters to search over
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'optimizer': [optim.Adam],
    'criterion': [nn.CrossEntropyLoss],
    'epochs': [30, 50],
    'batch_size': [4, 16, 32, 64],
    'patience': [5, 10, 15],
    'weight_decay': [0, 1e-4, 1e-2],
}


def get_optimizer(optimizer_class, parameters, lr, weight_decay, momentum=None):
    if optimizer_class == optim.Adam:
        return optim.Adam(parameters, lr=lr, weight_decay=weight_decay)
    elif optimizer_class == optim.SGD:
        # Ensure momentum is provided for SGD; otherwise, default to 0
        return optim.SGD(parameters, lr=lr, momentum=momentum if momentum is not None else 0, weight_decay=weight_decay)


def get_criterion(criterion_class):
    if criterion_class == nn.CrossEntropyLoss:
        return nn.CrossEntropyLoss()
    elif criterion_class == nn.NLLLoss:
        return nn.NLLLoss()


max_val_acc = -np.inf
combinations = list(product(*param_grid.values()))
best_params = None
for combination in tqdm(combinations):
    lr, optimizer_class, criterion_class, epochs, batch_size, patience, wd = combination
    
    dataloaders = prepare_datasets_and_loaders(df, feature_columns, batch_size=batch_size)
    
    # Initialize models, optimizers, and criterion for each combination
    model_vis = VisualClassifier()
    model_aud = AudialClassifier()

    model_vis = model_vis.to(device)
    model_aud = model_aud.to(device)

    optimizer_vis = get_optimizer(optimizer_class, model_vis.parameters(), lr, wd )
    optimizer_aud = get_optimizer(optimizer_class, model_aud.parameters(), lr, wd)
    criterion = get_criterion(criterion_class)
    
    # Training visual model
    print(f"Training Visual Model with lr={lr}, optimizer={optimizer_class.__name__}, criterion={criterion_class.__name__}, epochs={epochs}, batch_size={batch_size}, Patience={patience}, Weight decay={wd} ")
    val_acc = train_model(model_vis, {'train': dataloaders['extracted_visual_features_train'], 'val': dataloaders['extracted_visual_features_val']}, optimizer_vis, criterion, num_epochs=epochs, patience=patience, device=device)
    
    # Update best_params if current combination gives better validation accuracy
    if val_acc > max_val_acc:
        max_val_acc = val_acc
        best_params = {
            'learning_rate': lr,
            'optimizer': optimizer_class.__name__,
            'criterion': criterion_class.__name__,
            'epochs': epochs,
            'batch_size': batch_size,
            'patience': patience,
            'weight_decay': wd,
            'validation_accuracy': val_acc  # Include validation accuracy for reference
        }

    
    
    # # Training audio model
    # print(f"Training Audio Model with lr={lr}, optimizer={optimizer_class.__name__}, criterion={criterion_class.__name__}, epochs={epochs}, batch_size={batch_size}")
    # train_model(model_aud, {'train': dataloaders['extracted_audio_features_train'], 'val': dataloaders['extracted_audio_features_val']}, optimizer_aud, criterion, num_epochs=epochs)

    # Note: Add any additional logging or saving of model performance metrics as needed


  0%|          | 0/216 [00:00<?, ?it/s]

Training Visual Model with lr=0.001, optimizer=Adam, criterion=CrossEntropyLoss, epochs=30, batch_size=4, Patience=5, Weight decay=0 


  0%|          | 1/216 [00:23<1:25:29, 23.86s/it]

Early stopping triggered
Epoch 19, Train Loss: 1.1110, Train Accuracy: 0.7360, Val Loss: 1.2029, Val Accuracy: 0.4664
Training Visual Model with lr=0.001, optimizer=Adam, criterion=CrossEntropyLoss, epochs=30, batch_size=4, Patience=5, Weight decay=0.0001 


  1%|          | 2/216 [00:39<1:06:58, 18.78s/it]

...

Early stopping triggered
Epoch 30, Train Loss: 0.5474, Train Accuracy: 0.8324, Val Loss: 1.3412, Val Accuracy: 0.5037
Training Visual Model with lr=0.1, optimizer=Adam, criterion=CrossEntropyLoss, epochs=50, batch_size=64, Patience=15, Weight decay=0.0001 


100%|█████████▉| 215/216 [54:12<00:01,  1.90s/it]

Early stopping triggered
Epoch 28, Train Loss: 1.1872, Train Accuracy: 0.5543, Val Loss: 1.2880, Val Accuracy: 0.4366
Training Visual Model with lr=0.1, optimizer=Adam, criterion=CrossEntropyLoss, epochs=50, batch_size=64, Patience=15, Weight decay=0.01 


100%|██████████| 216/216 [54:14<00:00, 15.07s/it]

Early stopping triggered
Epoch 23, Train Loss: 1.3329, Train Accuracy: 0.3923, Val Loss: 1.3257, Val Accuracy: 0.3918


In [18]:
best_params

{'learning_rate': 0.001,
 'optimizer': 'Adam',
 'criterion': 'CrossEntropyLoss',
 'epochs': 30,
 'batch_size': 16,
 'patience': 15,
 'weight_decay': 0,
 'validation_accuracy': 0.5634328358208955}